In [ ]:
import datetime
from google.colab import files
from google.colab import drive
from datasets import load_dataset
import os
from sentence_transformers import SentenceTransformer, models, losses
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
from transformers import get_scheduler
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split, Dataset
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from tqdm import tqdm
from tqdm.auto import tqdm
import time
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psutil  # Для мониторинга памяти
import nltk
from nltk.corpus import stopwords
import re
import random

# Метрики качества кластеризации и предварительное тестирование моделей до обучения





# sergeyzh/rubert-mini-sts

In [ ]:
!pip install transformers sentencepiece scikit-learn matplotlib numpy

In [ ]:
file_path = r'/content/Гайки.xlsx'
df = pd.read_excel(file_path)

In [ ]:
# Извлекаем тексты из DataFrame
str_list = df.iloc[0:5]['text'].astype(str).tolist()

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list)}")
print("Пример первого текста:", str_list[0][:100] + "...")

Количество текстов: 5
Пример первого текста: 
Гайки (РИА Новости).
С.-ПЕТЕРБУРГ, 7 апр - РИА Новости. Ковка заготовок для корпуса реактора шестог...


In [ ]:
# Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("sergeyzh/rubert-mini-sts")
model = AutoModel.from_pretrained("sergeyzh/rubert-mini-sts")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sergeyzh/rubert-mini-sts")
model = AutoModel.from_pretrained("sergeyzh/rubert-mini-sts")


def embed_bert_cls(str_list, model, tokenizer):
    t = tokenizer(str_list, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

print(embed_bert_cls(str_list, model, tokenizer).shape)

(5, 312)


In [ ]:
# Получаем результат
embeddings = embed_bert_cls(str_list, model, tokenizer)

# Проверяем тип и свойства объекта
print("Тип объекта:", type(embeddings))
print("Размерность (shape):", embeddings.shape)
print("Тип данных элементов:", embeddings.dtype)
print("Пример данных:", embeddings[:5])

Тип объекта: <class 'numpy.ndarray'>
Размерность (shape): (5, 312)
Тип данных элементов: float32
Пример данных: [[ 0.0090804  -0.01375435 -0.03555515 ...  0.05024368 -0.01148374
   0.03677861]
 [ 0.03686041  0.00280563 -0.00183074 ...  0.02248335 -0.02557635
  -0.00416895]
 [ 0.02781539  0.0288245  -0.01763437 ...  0.03332358 -0.04105367
  -0.0180523 ]
 [ 0.00835762 -0.01002527 -0.02085105 ...  0.03154266 -0.00656344
   0.02785341]
 [ 0.03043623  0.01480447 -0.01573961 ...  0.02596574  0.00063615
   0.00704283]]


In [ ]:
# Кластеризация DBSCAN
dbscan = DBSCAN(
    eps=0.3,
    min_samples=2,
    metric="cosine"
)


clusters = dbscan.fit_predict(embeddings)

In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1    Гайки (РИА Новости). С.-ПЕТЕРБУРГ, 7 апр - РИА Но... 0         
2   Гайки (Россия 24) Авторасшифровка средняя. Россия ... 0         
3   Гайки (Смотрим) очень короткий. Смотрим; 08.04.202... 0         
4   Гайки (Тасс) средний. ТАСС # Федеральные округа Ро... 0         
5   Гайки (Телеканал Санкт-Петербург) короткий. ПЕТЕРБ... 0         


# 5 тематик

In [ ]:
file_path_5_topics = r'/content/5 Тематик.xlsx'
df_topics = pd.read_excel(file_path_5_topics)

In [ ]:
df_topics.head(10)

,text
0,5 технологий (Ведомости) длинный лонгрид. # П...
1,"Филиппины (Известия) длинный. Москва, 10 апре..."
2,Филиппины (Известия) короткий. Манила заинтере...
3,Радиация (РИА Новости) средний. # Все новости...
4,"Юбилей (Россия 24) авторасшифровка. Москва, 8 ..."
5,"Юбилей и разрешение (Россия 24). Россия 24, М..."
6,"Радиация (ТАСС) средний. Москва, 1 апреля 2025..."
7,Руководитель (Большое Радио) средний. (big-ra...
8,Руководитель (Мурманский вестник) средний. (m...


In [ ]:
# Извлекаем тексты из DataFrame
str_list_topics = df_topics.iloc[0:100]['text'].astype(str).tolist()

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list_topics)}")
print("Пример первого текста:", str_list_topics[0][:100] + "...")

Количество текстов: 9
Пример первого текста: 5 технологий (Ведомости) длинный лонгрид.  # Партнерское приложение (vedomosti.ru), Москва, 4 апреля...


In [ ]:
def embed_bert_cls(str_list_topics, model, tokenizer):
    t = tokenizer(str_list_topics, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

print(embed_bert_cls(str_list_topics, model, tokenizer).shape)

(9, 312)


In [ ]:
# Получаем результат
embeddings = embed_bert_cls(str_list_topics, model, tokenizer)

# Проверяем тип и свойства объекта
print("Тип объекта:", type(embeddings))
print("Размерность (shape):", embeddings.shape)
print("Тип данных элементов:", embeddings.dtype)
print("Пример данных:", embeddings[:5])

Тип объекта: <class 'numpy.ndarray'>
Размерность (shape): (9, 312)
Тип данных элементов: float32
Пример данных: [[-0.0360464  -0.01931516  0.01138463 ... -0.04293364  0.01405946
  -0.00917618]
 [-0.00631521  0.010145   -0.02676808 ... -0.00965563  0.00613984
  -0.00833112]
 [-0.01119098  0.03889713 -0.00588466 ...  0.02992999 -0.04398566
  -0.03818605]
 [-0.02015707  0.01658297  0.01250393 ...  0.0312162  -0.00853266
   0.00153065]
 [ 0.0165112   0.04215242 -0.01078786 ...  0.03016303 -0.01493649
  -0.00258887]]


In [ ]:
# Кластеризация DBSCAN
dbscan = DBSCAN(
    eps=0.3,
    min_samples=2,
    metric="cosine"
)


clusters = dbscan.fit_predict(embeddings)

In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list_topics, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1   5 технологий (Ведомости) длинный лонгрид.  # Партн... 0         
2   Филиппины (Известия) длинный.  Москва, 10 апреля 2... 0         
3   Филиппины (Известия) короткий. Манила заинтересова... 0         
4   Радиация (РИА Новости) средний.  # Все новости, Мо... 0         
5   Юбилей (Россия 24) авторасшифровка. Москва, 8 апре... 0         
6    Юбилей и разрешение (Россия 24). Россия 24, Москв... 0         
7   Радиация (ТАСС) средний. Москва, 1 апреля 2025 14:... 0         
8   Руководитель (Большое Радио) средний.  (big-radio.... 0         
9   Руководитель (Мурманский вестник) средний.  (mvest... 0         


# Контейнеры и музей

In [ ]:
file_path_ph = r'/content/контейнеры и физика.xlsx'
df_topics_container = pd.read_excel(file_path_ph)

In [ ]:
# Извлекаем тексты из DataFrame
str_list_container = df_topics_container.iloc[0:100]['text'].astype(str).tolist()

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list_container)}")
print("Пример первого текста:", str_list_container[0][:100] + "...")

Количество текстов: 8
Пример первого текста: Контейнеры (Infopro54 ru Новосибирск 15 апреля 2025 14:30) .
Они сократят потребность предприятия в ...


In [ ]:
def embed_bert_cls(str_list_container, model, tokenizer):
    t = tokenizer(str_list_container, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

print(embed_bert_cls(str_list_container, model, tokenizer).shape)

(8, 312)


In [ ]:
# Получаем результат
embeddings = embed_bert_cls(str_list_container, model, tokenizer)

# Проверяем тип и свойства объекта
print("Тип объекта:", type(embeddings))
print("Размерность (shape):", embeddings.shape)
print("Тип данных элементов:", embeddings.dtype)
print("Пример данных:", embeddings[:5])

Тип объекта: <class 'numpy.ndarray'>
Размерность (shape): (8, 312)
Тип данных элементов: float32
Пример данных: [[ 0.03942551 -0.0057153   0.00895876 ...  0.03879276 -0.0012283
  -0.0019963 ]
 [ 0.03242306  0.00754299  0.00334555 ...  0.05071279 -0.00630706
  -0.02149578]
 [ 0.07412617 -0.00979219  0.02765524 ...  0.02395076 -0.05222011
  -0.01480172]
 [ 0.05173521  0.00629294  0.02310496 ...  0.05205533 -0.03499265
  -0.00813912]
 [ 0.06920866  0.00502075 -0.00659214 ...  0.06339616 -0.01557503
  -0.0368683 ]]


In [ ]:
# Кластеризация DBSCAN
dbscan = DBSCAN(
    eps=0.3,
    min_samples=2,
    metric="cosine"
)


clusters = dbscan.fit_predict(embeddings)

In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list_container, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1   Контейнеры (Infopro54 ru Новосибирск 15 апреля 202... 0         
2   Контейнеры (Коммерсант). Автор: Кичан Михаил Сибир... 0         
3   Музей (Life.ru). В Музее "Атом" на ВДНХ открылся к... 0         
4   Музей(ТВ) центр - очень короткий. Технологии, за к... 0         
5   Музей (Смотрим) супер короткий. Класс квантовой фи... 0         
6   Музйе (россия 24) Авторасшифровка. Класс квантовой... 0         
7   Музей (Тасс) Средний. МОСКВА, 14 апреля. /ТАСС/. К... 0         
8   Музей (Тасс) Полный дубль. МОСКВА, 14 апреля.  Нов... 0         


# DeepPavlov/rubert-base-cased

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased'
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
file_path = r'/content/Гайки.xlsx'
df = pd.read_excel(file_path)

In [ ]:
# Извлекаем тексты из DataFrame
str_list = df.iloc[0:5]['text'].astype(str).tolist()

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list)}")
print("Пример первого текста:", str_list[0][:100] + "...")

Количество текстов: 5
Пример первого текста: 
Гайки (РИА Новости).
С.-ПЕТЕРБУРГ, 7 апр - РИА Новости. Ковка заготовок для корпуса реактора шестог...


In [ ]:
texts = df.iloc[0:5]['text']

In [ ]:
# Загружаем модель
model = SentenceTransformer('DeepPavlov/rubert-base-cased')

# Извлекаем тексты из DataFrame и преобразуем в строки
str_list = df.iloc[0:5]['text'].astype(str).tolist()
str_list = [text for text in str_list if text is not None and text.strip() != '']

# Получаем эмбеддинги
embeddings = model.encode(str_list)

# Выводим информацию о полученных эмбеддингах
print(f"Получено эмбеддингов: {embeddings.shape[0]}")
print(f"Размерность каждого эмбеддинга: {embeddings.shape[1]}")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Получено эмбеддингов: 5
Размерность каждого эмбеддинга: 768


In [ ]:
# Кластеризация DBSCAN
dbscan = DBSCAN(
    eps=0.3,
    min_samples=2,
    metric="cosine"
)


clusters = dbscan.fit_predict(embeddings)

In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1    Гайки (РИА Новости). С.-ПЕТЕРБУРГ, 7 апр - РИА Но... 0         
2   Гайки (Россия 24) Авторасшифровка средняя. Россия ... 0         
3   Гайки (Смотрим) очень короткий. Смотрим; 08.04.202... -1        
4   Гайки (Тасс) средний. ТАСС # Федеральные округа Ро... 0         
5   Гайки (Телеканал Санкт-Петербург) короткий. ПЕТЕРБ... 0         


# 5 тематик

In [ ]:
file_path_5_topics = r'/content/5 Тематик.xlsx'
df_topics = pd.read_excel(file_path_5_topics)

In [ ]:
# Извлекаем тексты из DataFrame
str_list_topics = df_topics.iloc[0:100]['text'].astype(str).tolist()

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list_topics)}")
print("Пример первого текста:", str_list_topics[0][:100] + "...")

Количество текстов: 9
Пример первого текста: 5 технологий (Ведомости) длинный лонгрид.  # Партнерское приложение (vedomosti.ru), Москва, 4 апреля...


In [ ]:
# Получаем эмбеддинги
embeddings = model.encode(str_list_topics)

# Выводим информацию о полученных эмбеддингах
print(f"Получено эмбеддингов: {embeddings.shape[0]}")
print(f"Размерность каждого эмбеддинга: {embeddings.shape[1]}")

Получено эмбеддингов: 9
Размерность каждого эмбеддинга: 768


In [ ]:
# Кластеризация DBSCAN
dbscan = DBSCAN(
    eps=0.3,
    min_samples=2,
    metric="cosine"
)

# Важно: передаем матрицу расстояний, а не сходств!
clusters = dbscan.fit_predict(embeddings)

In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list_topics, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1   5 технологий (Ведомости) длинный лонгрид.  # Партн... 0         
2   Филиппины (Известия) длинный.  Москва, 10 апреля 2... 0         
3   Филиппины (Известия) короткий. Манила заинтересова... 0         
4   Радиация (РИА Новости) средний.  # Все новости, Мо... 0         
5   Юбилей (Россия 24) авторасшифровка. Москва, 8 апре... 0         
6    Юбилей и разрешение (Россия 24). Россия 24, Москв... 0         
7   Радиация (ТАСС) средний. Москва, 1 апреля 2025 14:... 0         
8   Руководитель (Большое Радио) средний.  (big-radio.... 0         
9   Руководитель (Мурманский вестник) средний.  (mvest... 0         


# Контейнеры и музей

In [ ]:
file_path_ph = r'/content/контейнеры и физика.xlsx'
df_topics_container = pd.read_excel(file_path_ph)

In [ ]:
# Извлекаем тексты из DataFrame
str_list_container = df_topics_container.iloc[0:100]['text'].astype(str).tolist()

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list_container)}")
print("Пример первого текста:", str_list_container[0][:100] + "...")

Количество текстов: 8
Пример первого текста: Контейнеры (Infopro54 ru Новосибирск 15 апреля 2025 14:30) .
Они сократят потребность предприятия в ...


In [ ]:
# Получаем эмбеддинги
embeddings = model.encode(str_list_container)

# Выводим информацию о полученных эмбеддингах
print(f"Получено эмбеддингов: {embeddings.shape[0]}")
print(f"Размерность первого эмбеддинга: {embeddings.shape[1]}")

Получено эмбеддингов: 8
Размерность первого эмбеддинга: 768


In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list_container, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1   Контейнеры (Infopro54 ru Новосибирск 15 апреля 202... 0         
2   Контейнеры (Коммерсант). Автор: Кичан Михаил Сибир... 0         
3   Музей (Life.ru). В Музее "Атом" на ВДНХ открылся к... 0         
4   Музей(ТВ) центр - очень короткий. Технологии, за к... 0         
5   Музей (Смотрим) супер короткий. Класс квантовой фи... 0         
6   Музйе (россия 24) Авторасшифровка. Класс квантовой... 0         
7   Музей (Тасс) Средний. МОСКВА, 14 апреля. /ТАСС/. К... 0         
8   Музей (Тасс) Полный дубль. МОСКВА, 14 апреля.  Нов... 0         


# Подготовка данных и обучение rubert-mini-sts

In [ ]:
file_path = r'/content/тематики.csv'
df_main = pd.read_csv(file_path)

In [ ]:
df_main.shape

(24510, 3)

In [ ]:
df_main_clean = df_main.drop('title', axis = 1) #удаляем колонку с названиями текстов

In [ ]:
df_main_clean.shape

(24510, 2)

In [ ]:
# 1. Извлечение уникальных меток тематик
# Метод unique() возвращает массив уникальных значений из колонки 'title.1'

unique_labels = df_main_clean['title.1'].unique()
print(f"Уникальные метки: {unique_labels}")

# 2. Создание словаря для соответствия строковых меток числовым значениям
label_to_num = {}
current_number = 0  # Счётчик для числовых меток

# Итерация по уникальным меткам в порядке их появления
for label in unique_labels:
    if label not in label_to_num:
        # Присваиваем новой метке текущий номер
        label_to_num[label] = current_number
        current_number += 1  # Увеличиваем счётчик для следующей уникальной метки

print(f"Словарь соответствия: {label_to_num}")

# 3. Замена строковых меток на числовые в исходном DataFrame
# Метод map() заменяет значения в колонке на основе словаря

df_main_clean['numeric_label'] =df_main_clean['title.1'].map(label_to_num)

# Проверка результата
print("\nРезультат преобразования:")
print(df_main_clean[['title.1', 'numeric_label']].head())

Уникальные метки: ['(00) 2024.06.02 (01) Участники программы "Время Героев" отправились на атомном ледоколе к Северному полюсу'
 '(00) 2024.06.03 (01) Росатом запустил первую очередь крупнейшего в Поволжье центра обработки данных "Иннополис"'
 '(00) 2024.06.03 (02) Росатом и Республика Татарстан расширяют технологическое сотрудничество'
 ...
 '(12) 2024.06.07 (01) "Росатом" и Газпром будут вместе развивать проекты газомоторной отрасли'
 '(12) 2024.06.25 (01) Росатом разработал стеклоткань для нового паралета Федора Конюхова'
 '(13) 2024.06.18 (01) Представитель "Росатома" принял участие в международной выставке и форуме RENWEX-2024']
Словарь соответствия: {'(00) 2024.06.02 (01) Участники программы "Время Героев" отправились на атомном ледоколе к Северному полюсу': 0, '(00) 2024.06.03 (01) Росатом запустил первую очередь крупнейшего в Поволжье центра обработки данных "Иннополис"': 1, '(00) 2024.06.03 (02) Росатом и Республика Татарстан расширяют технологическое сотрудничество': 2, '(00)

In [ ]:
df_main_clean.head(40)

In [ ]:
df_new = df_main_clean.drop('title.1', axis = 1) #удаляем столбец с со словесными метками тематик

In [ ]:
df_new.shape

(24510, 2)

In [ ]:
# Список цитат, которые мы хотим найти и удалить
quotes_to_remove = ['Здесь лежали цитаты из файлов Гайки, 5 тематик и контейнеры и музеи']

# Функция для проверки, содержит ли строка цитату
def contains_quote(text, quotes):
    return any(quote in text for quote in quotes)

# Применяем фильтрацию строк с цитатами
df_new = df_new[~df_new['text'].apply(lambda x: contains_quote(x, quotes_to_remove))]


# Выводим очищенный DataFrame
print(df_new)

                                                    text  numeric_label
0      МОСКВА, 3 июня. /ТАСС/. Росатом поделится упра...              0
1      Первые 83 участника программы "Время героев" п...              0
2      Участники программы "Время героев" отправились...              0
3      Гендиректор госкорпорации Алексей Лихачев увер...              0
4      Теги: Кадровая программа Время Героев, ГК по а...              0
...                                                  ...            ...
24505  МОСКВА, 18 июн - РИА Новости. "Росатом" при по...           3275
24506  МОСКВА, 18 июн /ПРАЙМ/. "Росатом" при поддержк...           3275
24507  МОСКВА, 18 июн - РИА Новости. "Росатом" при по...           3275
24508  МОСКВА, 18 июн - РИА Новости. "Росатом" при по...           3275
24509  МОСКВА, 18 июн - РИА Новости. "Росатом" при по...           3275

[24461 rows x 2 columns]


In [ ]:
num_classes = df_new['numeric_label'].nunique()
print(f'Количество классов: {num_classes}')

Количество классов: 3273


In [ ]:
# Посмотрим, какие уникальные значения у нас есть в numeric_label
print("Уникальные значения в numeric_label:")
print(df_new['numeric_label'].unique())

# Отсортируем их для удобства
unique_labels = sorted(df_new['numeric_label'].unique())
print(f"\nОтсортированные уникальные метки: {unique_labels}")

# Определим количество классов
num_classes = len(unique_labels)
print(f"\nКоличество классов (num_classes): {num_classes}")

# Проверим диапазон меток
min_label = min(unique_labels)
max_label = max(unique_labels)
print(f"Минимальная метка: {min_label}")
print(f"Максимальная метка: {max_label}")

# Проверим, идут ли метки подряд от 0
expected_labels = list(range(num_classes))
print(f"Ожидаемые метки (если с 0): {expected_labels}")
print(f"Совпадают ли реальные метки с ожидаемыми: {unique_labels == expected_labels}")

Уникальные значения в numeric_label:
[   0    1    2 ... 3273 3274 3275]

Отсортированные уникальные метки: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.in

In [ ]:
text_from_df = df_new['text']
print(text_from_df)

0        МОСКВА, 3 июня. /ТАСС/. Росатом поделится упра...
1        Первые 83 участника программы "Время героев" п...
2        Участники программы "Время героев" отправились...
3        Гендиректор госкорпорации Алексей Лихачев увер...
4        Теги: Кадровая программа Время Героев, ГК по а...
                               ...                        
24505    МОСКВА, 18 июн - РИА Новости. "Росатом" при по...
24506    МОСКВА, 18 июн /ПРАЙМ/. "Росатом" при поддержк...
24507    МОСКВА, 18 июн - РИА Новости. "Росатом" при по...
24508    МОСКВА, 18 июн - РИА Новости. "Росатом" при по...
24509    МОСКВА, 18 июн - РИА Новости. "Росатом" при по...
Name: text, Length: 24461, dtype: object


In [ ]:
# Функция для очистки текста
def clean_text(text_from_df):
    # Удаляем все ссылки (с http, https, ftp и т.д.)
    text = re.sub(r'http\S+', '', text_from_df)  # удаляем ссылки типа http://, https:// и т.п.

    # Удаляем HTML-теги
    html = re.compile(r'<.*?>')
    text = html.sub(r'', text)  # удаляем HTML

    return text

# Пример текста с ссылкой
text = 'Это тестовый текст с ссылкой http://example.com и другим текстом http://another-link.com.'

# Очистка текста
cleaned_text = clean_text(text)

# Выводим очищенный текст
print(cleaned_text)



Это тестовый текст с ссылкой  и другим текстом 


In [ ]:
# Пример текста для очистки
text = '<html><body>Это тестовый текст с ссылкой http://example.com и другим http текстом.</body></html>'
cleaned_text = clean_text(text)

# Выводим очищенный текст
print(cleaned_text)

Это тестовый текст с ссылкой  и другим http текстом.


In [ ]:
df_new['text'] = df_new['text'].apply(lambda x: clean_text(x))

In [ ]:
# Сохраняем очищенный DataFrame в CSV файл
df_new.to_csv('тематики_2.csv', index=False, encoding='utf-8')

In [ ]:
# Сохраняем DataFrame в CSV файл
df_new.to_csv('тематики_2.csv', index=False, encoding='utf-8')

# Скачиваем файл
files.download('тематики_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''
Сохраняем очищенный текст и его лейблы и две переменные, которые на выходе представляют собой массив типа ndarray
'''

texts = df_new.text.values[:10]
labels = df_new.numeric_label.values[:10]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sergeyzh/rubert-mini-sts")
model = AutoModel.from_pretrained("sergeyzh/rubert-mini-sts")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
max_length = 2048  # Максимальная длина для модели


for idx, sent in enumerate(texts):
    # Токенизируем текст
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Обрезаем последовательность, если она слишком длинная
    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]

print(f" After truncation: {len(input_ids)}")
print(texts)

In [ ]:
input_ids = []
attention_masks = []


for text in texts:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        truncation=True,
                        max_length = max_length,
                        padding='max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


print('Original: ', texts)
print('Token IDs:', input_ids)
#print('Label:', labels.item())

In [ ]:

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))

val_size = len(dataset)  - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

    8 training samples
    2 validation samples


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e5fb51bea09e68aded08c80fa5ba4d6f7cd827a84457ea9242b8111e88aed7f7
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [ ]:
!pip install AutoTokenizer

ERROR: Could not find a version that satisfies the requirement AutoTokenizer (from versions: none)
ERROR: No matching distribution found for AutoTokenizer


In [ ]:
#ФИНАЛЬНАЯ ВЕРСИЯ

tokenizer = AutoTokenizer.from_pretrained("sergeyzh/rubert-mini-sts")
model = AutoModel.from_pretrained("sergeyzh/rubert-mini-sts")  # ИЗМЕНЕНИЕ ЗДЕСЬ: Используем AutoModel для эмбеддингов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

texts = df_new.text.values[:10]
labels = df_new.numeric_label.values[:10]

max_length = 2048

input_ids = []
attention_masks = []

for text in texts:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f'{train_size} training samples')
print(f'{val_size} validation samples')

# Класс для подбора размера батча
class BatchSizeSelector:
    def __init__(self, model, device, max_length):
        self.model = model
        self.device = device
        self.max_length = max_length
        # Создаем фиктивные данные для тестирования
        self.dummy_input_ids = torch.ones((1, max_length), dtype=torch.long)
        self.dummy_attention_mask = torch.ones((1, max_length), dtype=torch.long)

    def generate_batch(self, size):
        batch_size, sequence_len = size
        input_ids = self.dummy_input_ids.repeat(batch_size, 1)
        attention_mask = self.dummy_attention_mask.repeat(batch_size, 1)
        return {
            'input_ids': input_ids.to(self.device),
            'attention_mask': attention_mask.to(self.device),
        }

    def try_size_cpu(self, size):
        try:
            batch = self.generate_batch(size)
            outputs = self.model(**batch)
            vm = psutil.virtual_memory()
            if vm.percent >= 85:
                return False
            else:
                return True
        except RuntimeError as e:
            if 'out of memory' in str(e).lower():
                return False
            raise e

    def try_size_cuda(self, size):
        try:
            torch.cuda.empty_cache()
            batch = self.generate_batch(size)
            outputs = self.model(**batch)
            torch.cuda.empty_cache()
            return True
        except RuntimeError as e:
            if 'out of memory' in str(e).lower():
                torch.cuda.empty_cache()
                return False
            raise e

    def get_optimal_size(self):
        batch_size = 1
        sequence_len = self.max_length
        self.model.train()

        while True:
            size = (batch_size, sequence_len)

            if self.device.type == 'cuda':
                success = self.try_size_cuda(size)
            else:
                success = self.try_size_cpu(size)

            if success:
                batch_size *= 2
            else:
                batch_size //= 2
                return batch_size

# Используем BatchSizeSelector для подбора оптимального размера батча
batch_selector = BatchSizeSelector(
    model=model,
    device=device,
    max_length=max_length
)

batch_size = batch_selector.get_optimal_size()
print(f"Optimal batch size: {batch_size}")

# Инициализация DataLoader'ов с автоматически подобранным размером батча
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)
val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)

# Класс для обучения модели и извлечения эмбеддингов
class Trainer:
    def __init__(self, model, epochs=10):
        self.model = model
        self.epochs = epochs
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

        self.losses = []  # Для отслеживания потерь

    def evaluate(self, dataloader):
        self.model.eval()
        embeddings = []  # Сюда будем собирать эмбеддинги

        for batch in tqdm(dataloader, desc="Evaluating"):  # Прогресс бар для валидации
            batch = tuple(t.to(self.device) for t in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1]
            }
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Среднее по всем токенам в предложении
            embeddings.append(outputs.last_hidden_state.mean(dim=1).cpu().numpy())

        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

    def fit(self, train_dataloader, valid_dataloader, save_path):
        optimizer = AdamW(self.model.parameters(), lr=5e-5)
        total_steps = len(train_dataloader) * self.epochs
        scheduler = get_scheduler(
            'linear',
            optimizer=optimizer,
            num_warmup_steps=0,
            num_training_steps=total_steps
        )

        best_loss = np.inf
        for epoch in range(self.epochs):
            self.model.train()
            total_loss = 0

            for batch in tqdm(train_dataloader, desc=f"Training epoch {epoch+1}/{self.epochs}"):  # Прогресс бар для обучения
                batch = tuple(t.to(self.device) for t in batch)
                inputs = {
                    'input_ids': batch[0],
                    'attention_mask': batch[1]
                }
                self.model.zero_grad()
                outputs = self.model(**inputs)
                loss = outputs.loss
                total_loss += loss.item()
                loss.backward()
                clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()

            avg_train_loss = total_loss / len(train_dataloader)
            eval_embeddings = self.evaluate(valid_dataloader)
            avg_val_loss = np.mean([np.linalg.norm(embed) for embed in eval_embeddings])  # Пример метрики качества эмбеддингов

            self.losses.append((avg_train_loss, avg_val_loss))

            # Сохраняем модель после каждой эпохи
            torch.save({
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, save_path + f'best_model_epoch_{epoch+1}.pt')  # Путь для сохранения модели

            print(f'Epoch {epoch+1}/{self.epochs}')
            print(f'Train loss: {avg_train_loss:.3f} Val loss: {avg_val_loss:.3f}')

        self.plot_loss()  # Построение графика

    def plot_loss(self):
        # График потерь
        train_losses, val_losses = zip(*self.losses)
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()  # Показываем график

# Инициализация и обучение
trainer = Trainer(model=model, epochs=3)
trainer.fit(train_dataloader, val_dataloader, save_path='/content/model')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NameError: name 'df_new' is not defined

# Обучение с AllTripletLoss

In [ ]:
file_path = r'/content/тематики_2.csv'
df_new = pd.read_csv(file_path)

Датафрейм теперь разделен на предложения. До этого в колонке text лежали строки с целыми текстами, каждому из которых была присвоена тематика в числовом формате. Теперь в датафреймы лежат строки с теми же метками класса, что и их тексты. Меток тематик - 3275, предложений 460 428

In [ ]:
# Создаем пустые списки для предложений и меток
sentences = []
labels = []

# Итерация по строкам исходного DataFrame
current_label = None  # Изначально нет метки

for idx, row in df_new.iterrows():
    text = row['text']  # Получаем текст
    label = row['numeric_label']  # Получаем метку для этого текста

    # Если метка изменяется, обновляем current_label
    if label != current_label:
        current_label = label  # Обновляем метку, если она изменилась

    # Разделяем текст на предложения с помощью регулярных выражений
    split_text = re.split(r'(?<=[.!?])\s+', text)  # Разделяем по точке, вопросительному или восклицательному знаку с пробелом после них

    # Для каждого предложения сохраняем исходную метку
    for s in split_text:
        s = s.strip()  # Удаляем лишние пробелы
        if s:  # Пропускаем пустые строки
            sentences.append(s)  # Добавляем предложение в список
            labels.append(current_label)  # Добавляем текущую метку для этого предложения

# Теперь мы создаем новый DataFrame с предложениями и метками
df_new = pd.DataFrame({
    'Text': sentences,
    'Label': labels
})

# Печать результата
print(df_new.head())

In [ ]:
# Переименовываем столбцы для последующей передачи в лосс
df_new = df_new.rename(columns={'text': 'Texts', 'numeric_label': 'Labels'})

In [ ]:

file_path = r'/content/тематики_3.csv'
df_new = pd.read_csv(file_path)

In [ ]:
# Сохраняем DataFrame в CSV файл
df_new.to_csv('тематики_3.csv', index=False, encoding='utf-8')

# Скачиваем файл
files.download('тематики_3.csv')

In [ ]:
print(df_new.columns)

Index(['Text', 'Label'], dtype='object')


In [ ]:
df_new.shape

(460427, 2)

In [ ]:
df_new = df_new.iloc[:100000]



In [ ]:
df_new.shape

(100000, 2)

In [ ]:

# Сохраняем DataFrame в CSV файл
df_new.to_csv('тематики_4_cut.csv', index=False, encoding='utf-8')

# Скачиваем файл
files.download('тематики_4_cut.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Путь к папке final_model на Google Drive, проверяем способность кода сохранить файл в директорию
final_model_path = '/content/drive/MyDrive/final_model/'


if not os.path.exists(final_model_path):
    os.makedirs(final_model_path)

# Сохраняем DataFrame в формате CSV
csv_path = os.path.join(final_model_path, 'df_new.csv')
df_new.to_csv(csv_path, index=False)
print(f'DataFrame сохранен как CSV: {csv_path}')

DataFrame сохранен как CSV: /content/drive/MyDrive/final_model/df_new.csv


In [ ]:
#последняя версия


# 1. Загрузка модели и отправка на GPU
model_name = 'DeepPavlov/rubert-base-cased'
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Отправляем модель на GPU
print(f"GPU доступен: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Устройство: {torch.cuda.get_device_name(0)}")

print(f"Доступен GPU: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model is loaded and moved to {device}.")

# 2. Подготовка данных
file_path = r'/content/тематики_4_cut.csv'
df_new = pd.read_csv(file_path)



# 3. Создание кастомного датасета
class TextsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


# Разделяем данные на тренировочные и валидационные (80% - 20%)
train_df, val_df = train_test_split(df_new, test_size=0.2, random_state=42)
train_dataset = TextsDataset(train_df['Text'].values, train_df['Label'].values, tokenizer)
val_dataset = TextsDataset(val_df['Text'].values, val_df['Label'].values, tokenizer)
print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")

# Создаем DataLoader'ы
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 4. Сохранение модели и токенизатора после каждой эпохи
def save_model_and_tokenizer(model, tokenizer, optimizer, epoch, save_path):
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, os.path.join(save_path, f'checkpoint_epoch_{epoch}.pt'))

    print(f"Model, tokenizer, and optimizer state saved to {save_path} at epoch {epoch}")

# 5. Сохранение графика потерь
def save_loss_plot(train_losses, val_losses, save_path):
    try:
        plt.figure(figsize=(10, 6))
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()

        plot_path = os.path.join(save_path, 'loss_plot.png')
        plt.savefig(plot_path)
        plt.close()

        print(f"Loss plot saved to {plot_path}")
    except Exception as e:
        print(f"Error saving loss plot: {e}")


# 6. Основной цикл обучения
epochs = 10
batch_size = 8
learning_rate = 1e-6
save_path = r'/content/drive/MyDrive/final_model/'

# Определяем оптимизатор и шедулер
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

# Функция потерь (BatchAllTripletLoss)
loss_function = losses.BatchAllTripletLoss(model)

# Массивы для отслеживания потерь
train_losses = []
val_losses = []

# 7. Обучение
for epoch in range(epochs):
    model.train()
    train_loss = 0  # Сумма потерь за эпоху

    # Прогресс-бар для тренировочного процесса
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch+1}/{epochs} Training', unit='batch') as pbar_train:
        for batch in train_dataloader:
            # Загружаем данные в GPU
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Создаем словарь с нужными параметрами
            inputs = {'input_ids': input_ids,
                     'attention_mask': attention_mask}

            # Обнуляем градиенты
            optimizer.zero_grad()

            # Вычисляем loss
            loss = loss_function([inputs], labels)

            # Обратное распространение и шаг оптимизатора
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            pbar_train.update(1)

        # Среднее значение потерь по эпохе для обучения
        avg_train_loss = train_loss / len(train_dataloader)  # Средний тренировочный лосс
        pbar_train.set_postfix(train_loss=avg_train_loss)

        # Записываем потери в список
        train_losses.append(avg_train_loss)

    # Валидация
    model.eval()
    val_loss = 0  # Сумма потерь за эпоху

    # Прогресс-бар для валидации
    with tqdm(total=len(val_dataloader), desc=f'Epoch {epoch+1}/{epochs} Validation', unit='batch') as pbar_val:
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                inputs = {'input_ids': input_ids,
                          'attention_mask': attention_mask}

                # Вычисляем loss для валидации
                loss = loss_function([inputs], labels)

                val_loss += loss.item()
                pbar_val.update(1)

        # Среднее значение потерь по эпохе для валидации
        avg_val_loss = val_loss / len(val_dataloader)  # Средний валидационный лосс
        pbar_val.set_postfix(val_loss=avg_val_loss)

        # Записываем потери в список
        val_losses.append(avg_val_loss)

    # Выводим потери для текущей эпохи
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}")

    # Сохраняем модель, токенизатор, оптимизатор после каждой эпохи
    # Путь к папке final_model на Google Drive
    save_path= '/content/drive/MyDrive/final_model/'

    # Убедимся, что папка существует (создадим если нет)
    if not os.path.exists( save_path):
      os.makedirs( save_path)
    save_model_and_tokenizer(model, tokenizer, optimizer, epoch + 1, save_path)


    # Сохраняем график потерь
    save_loss_plot(train_losses, val_losses, save_path)

    # Шедулер
    scheduler.step()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Epoch 2/10, Train Loss: 4.038350153231621, Validation Loss: 4.0573758115768435
Model, tokenizer, and optimizer state saved to /content/drive/MyDrive/final_model/ at epoch 2
Loss plot saved to /content/drive/MyDrive/final_model/loss_plot.png


Epoch 3/10 Validation: 100%|██████████| 625/625 [02:13<00:00,  4.68batch/s, val_loss=4.06]


Epoch 3/10, Train Loss: 3.971339302134514, Validation Loss: 4.055309675550461
Model, tokenizer, and optimizer state saved to /content/drive/MyDrive/final_model/ at epoch 3
Loss plot saved to /content/drive/MyDrive/final_model/loss_plot.png


Epoch 4/10 Training:   6%|▌         | 154/2500 [01:43<26:09,  1.50batch/s]


KeyboardInterrupt: 

Здесь модель была обучена на меньшем объеме данных в связи с ограниченностью доступа гпу. Такой же код применила на внутренних сервисах с помощью гпу, результата лосса после первой эпохи 1,68 и далее не убывал. Таким образом, самая удачная модель без переобучения - 1 эпоха.

# После обучения

In [ ]:
# Загрузка модели и токенизато
model_path = '/content/drive/MyDrive/final_model/checkpoint_epoch_2.pt'
save_path = '/content/drive/MyDrive/final_model/'
model = AutoModel.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)

In [ ]:
file_path_5_topics = r'/content/5 Тематик.xlsx'
df_topics = pd.read_excel(file_path_5_topics)

In [ ]:
# Извлекаем тексты из DataFrame
str_list_topics = df_topics.iloc[0:100]['text'].astype(str).tolist()

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list_topics)}")
print("Пример первого текста:", str_list_topics[0][:100] + "...")

Количество текстов: 9
Пример первого текста: 5 технологий (Ведомости) длинный лонгрид.  # Партнерское приложение (vedomosti.ru), Москва, 4 апреля...


In [ ]:
# Функция для получения эмбеддингов
def get_embeddings(str_list_topics ):
    input_ids = []
    attention_masks = []

    # Токенизация текста
    for text in str_list_topics:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=2048,  # Максимальная длина
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0).to(device)
    attention_masks = torch.cat(attention_masks, dim=0).to(device)

    with torch.no_grad():
        # Получаем выходные эмбеддинги из модели
        outputs = model(input_ids=input_ids, attention_mask=attention_masks)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Используем средний пуллинг
        return embeddings.cpu().numpy()

# Получаем эмбеддинги для текстов
embeddings = get_embeddings(str_list_topics )

NameError: name 'device' is not defined

In [ ]:
# Кластеризация DBSCAN
dbscan = DBSCAN(
    eps=0.8,
    min_samples=1,
    metric="cosine"
)

# Важно: передаем матрицу расстояний, а не сходств!
clusters = dbscan.fit_predict(embeddings)

In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list_topics, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1   5 технологий (Ведомости) длинный лонгрид.  # Партн... 0         
2   Филиппины (Известия) длинный.  Москва, 10 апреля 2... 0         
3   Филиппины (Известия) короткий. Манила заинтересова... 0         
4   Радиация (РИА Новости) средний.  # Все новости, Мо... 0         
5   Юбилей (Россия 24) авторасшифровка. Москва, 8 апре... 0         
6    Юбилей и разрешение (Россия 24). Россия 24, Москв... 0         
7   Радиация (ТАСС) средний. Москва, 1 апреля 2025 14:... 0         
8   Руководитель (Большое Радио) средний.  (big-radio.... 0         
9   Руководитель (Мурманский вестник) средний.  (mvest... 0         


In [ ]:
file_path_ph = r'/content/контейнеры и физика.xlsx'
df_topics_container = pd.read_excel(file_path_ph)

In [ ]:
# Извлекаем тексты из DataFrame
str_list_container = df_topics_container.iloc[0:100]['text'].astype(str).tolist()  # Замените 'text' на имя вашего столбца

# Проверяем количество извлеченных текстов
print(f"Количество текстов: {len(str_list_container)}")
print("Пример первого текста:", str_list_container[0][:100] + "...")

Количество текстов: 8
Пример первого текста: Контейнеры (Infopro54 ru Новосибирск 15 апреля 2025 14:30) .
Они сократят потребность предприятия в ...


In [ ]:
# Функция для получения эмбеддингов
def get_embeddings(str_list_container ):
    input_ids = []
    attention_masks = []

    # Токенизация текста
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=2048,  # Максимальная длина
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0).to(device)
    attention_masks = torch.cat(attention_masks, dim=0).to(device)

    with torch.no_grad():
        # Получаем выходные эмбеддинги из модели
        outputs = model(input_ids=input_ids, attention_mask=attention_masks)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Используем средний пуллинг
        return embeddings.cpu().numpy()

# Получаем эмбеддинги для текстов
embeddings = get_embeddings(str_list_container )

In [ ]:
# Кластеризация DBSCAN
dbscan = DBSCAN(
    eps=0.8,           # Максимальное расстояние между образцами одного кластера
    min_samples=1,     # Минимальное количество образцов в кластере
    metric="cosine"  # Используем предварительно вычисленную матрицу
)

# Важно: передаем матрицу расстояний, а не сходств!
clusters = dbscan.fit_predict(embeddings)

In [ ]:
# Выводим заголовок таблицы
print("{:<50} {:<10}".format("Первые 50 символов текста", "Кластер"))
print("-" * 65)

# Обрабатываем каждый текст и метку кластера
for i, (text, cluster_id) in enumerate(zip(str_list_container, clusters)):
    # Берем первые 50 символов текста
    truncated_text = text[:50]
    # Заменяем переносы строк на пробелы для компактного отображения
    truncated_text_clean = truncated_text.replace('\n', ' ')
    # Добавляем многоточие если текст был обрезан
    if len(text) > 50:
        truncated_text_clean += "..."

    # Выводим в два столбца с нумерацией строк
    print("{:<3} {:<50} {:<10}".format(i+1, truncated_text_clean, cluster_id))

Первые 50 символов текста                          Кластер   
-----------------------------------------------------------------
1   Контейнеры (Infopro54 ru Новосибирск 15 апреля 202... 0         
2   Контейнеры (Коммерсант). Автор: Кичан Михаил Сибир... 0         
3   Музей (Life.ru). В Музее "Атом" на ВДНХ открылся к... 0         
4   Музей(ТВ) центр - очень короткий. Технологии, за к... 0         
5   Музей (Смотрим) супер короткий. Класс квантовой фи... 0         
6   Музйе (россия 24) Авторасшифровка. Класс квантовой... 0         
7   Музей (Тасс) Средний. МОСКВА, 14 апреля. /ТАСС/. К... 0         
8   Музей (Тасс) Полный дубль. МОСКВА, 14 апреля.  Нов... 0         
